In [2]:
%pip install duckdb pandas python-dotenv requests boto3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import duckdb
import pandas as pd
import os
import requests
import urllib.request
import re
from dotenv import load_dotenv
from datetime import datetime, timedelta

LAKE_LAYER = 'bronze'

# Load environment variables from .env file
load_dotenv('../../.env', override=True) 

True

In [4]:
RUSTFS_HOST = os.getenv('RUSTFS_HOST', 'localhost')
RUSTFS_PORT = os.getenv('RUSTFS_PORT', '8080')
RUSTFS_USER = os.getenv('RUSTFS_USER', 'admin')
RUSTFS_PASSWORD = os.getenv('RUSTFS_PASSWORD', 'password')
RUSTFS_BUCKET = os.getenv('RUSTFS_BUCKET', 'mitma')
RUSTFS_SSL = os.getenv('RUSTFS_SSL', 'false')

# Postgres Configuration
POSTGRES_USER = os.getenv('POSTGRES_USER', 'postgres')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'password')
POSTGRES_HOST = os.getenv('POSTGRES_HOST', 'localhost')
POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')
POSTGRES_DB = os.getenv('POSTGRES_DB', 'muceim')

# Construct S3 Endpoint with protocol
S3_ENDPOINT = f"{RUSTFS_HOST}:{RUSTFS_PORT}"


In [5]:
print(f"Connecting to RustFS at {S3_ENDPOINT}...")

# Initialize DuckDB Connection
con = duckdb.connect()

# Install and Load extensions
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute("INSTALL postgres;")
con.execute("LOAD postgres;")
con.execute("INSTALL ducklake;")
con.execute("LOAD ducklake;")

# Configure S3 Secrets for RustFS
con.execute(f"SET s3_endpoint='{S3_ENDPOINT}';")
con.execute(f"SET s3_access_key_id='{RUSTFS_USER}';")
con.execute(f"SET s3_secret_access_key='{RUSTFS_PASSWORD}';")
con.execute(f"SET s3_use_ssl={RUSTFS_SSL};")
con.execute("SET s3_url_style='path';")
con.execute("SET preserve_insertion_order=false;")
con.execute("SET max_temp_directory_size='40GiB';")


# Attach DuckLake with Postgres Catalog
postgres_connection_string = f"dbname={POSTGRES_DB} host={POSTGRES_HOST} user={POSTGRES_USER} password={POSTGRES_PASSWORD} port={POSTGRES_PORT}"
attach_query = f"ATTACH 'ducklake:postgres:{postgres_connection_string}' AS ducklake (DATA_PATH 's3://{RUSTFS_BUCKET}/');"

print(f"Attaching DuckLake with query: {attach_query}")
con.execute(attach_query)
con.execute("USE ducklake;")

print("DuckLake configured with Postgres catalog and RustFS storage.")

Connecting to RustFS at localhost:9000...
Attaching DuckLake with query: ATTACH 'ducklake:postgres:dbname=mitma host=localhost user=admin password=muceim-duckduck.2025! port=30432' AS ducklake (DATA_PATH 's3://mitma/');
DuckLake configured with Postgres catalog and RustFS storage.


In [6]:
import boto3
from botocore.client import Config

s3 = boto3.resource('s3',
    endpoint_url=f'http://{RUSTFS_HOST}:{RUSTFS_PORT}',
    aws_access_key_id=RUSTFS_USER,
    aws_secret_access_key=RUSTFS_PASSWORD,
    config=Config(signature_version='s3v4'),
    verify=False
)

if (not s3.Bucket(RUSTFS_BUCKET).creation_date):
    try:
        s3.create_bucket(Bucket=RUSTFS_BUCKET)
        print(f"Bucket '{RUSTFS_BUCKET}' created.")
    except Exception as e:
        print(f"Error creating bucket: {e}")
    

In [7]:
import boto3
from botocore.client import Config

def cleanup_mitma_system():
    """
    Cleans up the MITMA system by resetting the metadata 
    in PostgreSQL and clearing the storage in RustFS.
    """
    print("\n[1/2] Resetting Metadata in PostgreSQL...")
    try:
        SQL("DROP SCHEMA public CASCADE;")
        SQL("CREATE SCHEMA public;")
        SQL("GRANT ALL ON SCHEMA public TO postgres;")
        SQL("GRANT ALL ON SCHEMA public TO public;")
        print("  ✓ Schema 'public' reseted.")
    except Exception as e:
        print(f"  ❌ Error resetting Postgres: {e}")
        return # Paramos si falla la DB

    print("\n[2/2] Resetting Storage in RustFS...")
    try:
        bucket = s3.Bucket(RUSTFS_BUCKET)

        if not bucket.creation_date:
            return
        
        # Borramos todo el contenido
        bucket.objects.all().delete()
        # Borramos el bucket
        bucket.delete()
        #recreamos el bucket
        s3.create_bucket(Bucket=RUSTFS_BUCKET)
        print(f"  ✓ Bucket '{RUSTFS_BUCKET}' recreated.")
    except Exception as e:
        print(f"  ❌ Error cleaning RustFS: {e}")

    print("\n✅ MITMA SYSTEM RESETED.")


# Uncomment to run cleanup:
# cleanup_mitma_system()

In [8]:
def SQL(query):
    """Execute a SQL query and return the result as a Pandas DataFrame."""
    try:
        return con.execute(query).fetchdf()
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

In [9]:
def get_mitma_urls(dataset, zone_type, start_date, end_date):
    """
    Fetches MITMA URLs from RSS feed and filters by dataset, zone type, and date range.
    """
    rss_url = "https://movilidad-opendata.mitma.es/RSS.xml"
    
    # Simple mapping: dataset -> (url_path, file_prefix)
    dataset_map = {
        "od": ("viajes", "Viajes"),
        "people_day": ("personas", "Personas_dia"),
        "overnight_stay": ("pernoctaciones", "Pernoctaciones")
    }
    
    if zone_type not in ["distritos", "municipios", "gau"]:
        raise ValueError(f"Invalid zone_type: {zone_type}. Must be 'distritos', 'municipios', or 'gau'.")
    if dataset not in dataset_map:
        raise ValueError(f"Invalid dataset: {dataset}. Must be one of {list(dataset_map.keys())}.")
    
    dataset_path, file_prefix = dataset_map[dataset]
    
    # Construct file pattern: {Prefix}_{zone} (GAU is uppercase in files)
    zone_suffix = "GAU" if zone_type == "gau" else zone_type
    file_pattern = f"{file_prefix}_{zone_suffix}"
    
    # Build dynamic regex pattern
    # Pattern: https://.../por-{zone}/viajes/ficheros-diarios/YYYY-MM/YYYYMMDD_{FilePattern}.csv.gz
    pattern = rf'(https?://[^\s"<>]*/estudios_basicos/por-{zone_type}/{dataset_path}/ficheros-diarios/\d{{4}}-\d{{2}}/(\d{{8}})_{file_pattern}\.csv\.gz)'
        
    # Fetch RSS with User-Agent to avoid 403
    req = urllib.request.Request(rss_url, headers={"User-Agent": "MITMA-DuckLake-Loader"})
    txt = urllib.request.urlopen(req).read().decode("utf-8", "ignore")
    
    # Find all matches (case-insensitive for por-gau vs por-GAU)
    matches = re.findall(pattern, txt, re.I)
    
    # Remove duplicates using set (RSS often has duplicate entries)
    unique_matches = list(set(matches))
    
    # Convert date range to comparable format
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Filter by date range and sort
    filtered_urls = []
    for url, date_str in unique_matches:
        file_date = datetime.strptime(date_str, "%Y%m%d")
        if start_dt <= file_date <= end_dt:
            filtered_urls.append((url, date_str))
    
    # Sort by date ascending
    filtered_urls.sort(key=lambda x: x[1])
    
    # Extract just the URLs
    urls = [url for url, _ in filtered_urls]
    
    print(f"Found {len(urls)} URLs for {dataset} {zone_type} from {start_date} to {end_date}")
    
    if not urls:
        print(f"WARNING: No URLs found. Check if data exists for the requested date range.")
    
    return urls

In [10]:
def create_and_merge_table(table_name, urls):
    """
    Generic function to create table and merge data for any MITMA dataset.
    Uses ALL columns from the CSV as merge keys (bronze layer pattern).

    Parameters:
    - dataset: 'od', 'people_day', 'overnight_stay'
    - zone_type: 'distritos', 'municipios', 'gau'
    - urls: list of URLs to load
    """
    
    table_name = f'{LAKE_LAYER}_{table_name}'
    
    # Convert list of URLs to a string representation for DuckDB list
    url_list_str = "[" + ", ".join([f"'{u}'" for u in urls]) + "]"

    # Step 1: Create table if not exists (using first file for schema inference)
    SQL(f"""
        CREATE TABLE IF NOT EXISTS {table_name} AS
        SELECT 
            * EXCLUDE (filename),
            CURRENT_TIMESTAMP AS loaded_at,
            filename AS source_file
        FROM read_csv(
            {url_list_str},
            filename = true,
            all_varchar = true
        )
        LIMIT 0;
    """)
    
    # Get column names from the table (excluding audit columns)
    columns_df = SQL(f"""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'
        AND column_name NOT IN ('loaded_at', 'source_file')
        ORDER BY ordinal_position;
    """)
    
    merge_keys = columns_df['column_name'].tolist()
    
    # Build ON clause from all CSV columns
    on_clause = " AND ".join([f"target.{key} = source.{key}" for key in merge_keys])
    
    # Step 3: MERGE for idempotent incremental loads
    SQL(f"""
        MERGE INTO {table_name} AS target
        USING (
            SELECT 
                * EXCLUDE (filename),
                CURRENT_TIMESTAMP AS loaded_at,
                filename AS source_file
            FROM read_csv(
                {url_list_str},
                filename = true,
                all_varchar = true
            )
        ) AS source
        ON {on_clause}
        WHEN MATCHED THEN
            UPDATE SET *
        WHEN NOT MATCHED THEN
            INSERT *;
    """)
    
    print(f"Table {table_name} merged successfully with {len(merge_keys)} key columns.")

In [127]:
def create_and_merge_table_from_json(table_name, urls, key_columns=None):
    """
    Generic function to create table and merge data from JSON API endpoints using DuckDB's read_json.
    
    Parameters:
    - table_name: Name of the table to create/merge into
    - urls: Single URL (string) or list of URLs that return JSON data (array of objects)
    - key_columns: List of column names to use as merge keys. If None, uses all columns.
    """
    
    table_name = f'{LAKE_LAYER}_{table_name}'
    
    # Normalize urls to list
    if isinstance(urls, str):
        urls = [urls]
    
    print(f"Fetching JSON data from {len(urls)} URL(s)...")
    
    # Step 1: Create table if not exists using DuckDB's read_json with first URL
    SQL(f"""
        CREATE TABLE IF NOT EXISTS {table_name} AS
        SELECT 
            *,
            CURRENT_TIMESTAMP AS loaded_at,
            '{urls[0]}' AS source_url
        FROM read_json('{urls[0]}', format='array')
        LIMIT 0;
    """)
    
    # Step 2: Get column names from the table (excluding audit columns)
    columns_df = SQL(f"""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'
        AND column_name NOT IN ('loaded_at', 'source_url')
        ORDER BY ordinal_position;
    """)
    
    data_columns = columns_df['column_name'].tolist()
    
    # Step 3: Determine merge keys
    if key_columns is None:
        merge_keys = data_columns
    else:
        merge_keys = key_columns
        # Validate that key columns exist
        missing_keys = [k for k in merge_keys if k not in data_columns]
        if missing_keys:
            raise ValueError(f"Key columns {missing_keys} not found in data. Available columns: {data_columns}")
    
    print(f"Using merge keys: {merge_keys}")
    
    # Step 4: Build ON clause
    on_clause = " AND ".join([f'target."{key}" = source."{key}"' for key in merge_keys])
    
    # Step 5: Build UNION ALL query for all URLs
    union_queries = []
    for url in urls:
        union_queries.append(f"""
            SELECT 
                *,
                CURRENT_TIMESTAMP AS loaded_at,
                '{url}' AS source_url
            FROM read_json('{url}', format='array')
        """)
    
    combined_source = "\nUNION ALL\n".join(union_queries)
    
    # Step 6: MERGE for idempotent incremental loads
    merge_query = f"""
        MERGE INTO {table_name} AS target
        USING (
            {combined_source}
        ) AS source
        ON {on_clause}
        WHEN MATCHED THEN
            UPDATE SET *
        WHEN NOT MATCHED THEN
            INSERT *;
    """
    
    SQL(merge_query)
    
    # Get row count
    count_result = SQL(f"SELECT COUNT(*) as count FROM {table_name}")
    row_count = count_result.iloc[0, 0]
    
    print(f"Table {table_name} merged successfully with {len(merge_keys)} key columns. Total rows: {row_count}")

In [12]:
def load_od_matrices(type="districts", start_date='2022-03-01', end_date='2022-03-03'):
    """
    Load OD matrices for the specified type and date range.
    """
    table_name = 'mitma_od'
    urls = get_mitma_urls(dataset, type, start_date, end_date)
    create_and_merge_table(table_name, urls)

In [13]:
def load_people_day(type="districts", start_date='2022-03-01', end_date='2022-03-03'):
    """
    Load people_day data for a specific type and date range.
    """
    table_name = 'mitma_people_day'
    urls = get_mitma_urls(dataset, type, start_date, end_date)
    create_and_merge_table(table_name, urls)

In [14]:
def load_overnight_stay(type="districts", start_date='2022-03-01', end_date='2022-03-03'):
    """
    Load overnight stay data for a specific type and date range.
    """
    table_name = 'mitma_overnight_stay'
    urls = get_mitma_urls(dataset, type, start_date, end_date)
    create_and_merge_table(table_name, urls)

In [15]:
# Verify connection
print("DuckDB Version:", SQL("SELECT version();").iloc[0,0])

# Check tables
print("Tables in DuckLake:")
print(SQL("SHOW TABLES;"))

DuckDB Version: v1.4.2
Tables in DuckLake:
                                      name
0                      bronze_ine_empresas
1            bronze_ine_empresas_municipio
2                    bronze_ine_municipios
3           bronze_ine_poblacion_municipio
4               bronze_ine_renta_municipio
5                   bronze_mitma_distritos
6                         bronze_mitma_gau
7               bronze_mitma_ine_relations
8         bronze_mitma_mitma_ine_relations
9                  bronze_mitma_municipios
10               bronze_mitma_od_distritos
11                     bronze_mitma_od_gau
12              bronze_mitma_od_municipios
13   bronze_mitma_overnight_stay_distritos
14         bronze_mitma_overnight_stay_gau
15  bronze_mitma_overnight_stay_municipios
16       bronze_mitma_people_day_distritos
17             bronze_mitma_people_day_gau
18      bronze_mitma_people_day_municipios
19                          ine_municipios
20             silver_mitma_overnight_stay
21         

In [16]:
%pip install requests geopandas

Note: you may need to restart the kernel to use updated packages.


In [17]:
import urllib.request
import re

def get_mitma_zoning_urls(zone_type):
    """
    Fetches MITMA Zoning URLs (Shapefiles + CSVs) from RSS feed using Regex.
    Matches the style of 'get_mitma_urls' but for static zoning files.
    """
    rss_url = "https://movilidad-opendata.mitma.es/RSS.xml"
    
    # Normalización de input
    if zone_type not in ["distritos", "municipios", "gau"]:
        raise ValueError(f"Invalid zone_type: {zone_type}. Must be 'distritos', 'municipios', or 'gau'.")

    # Lógica de sufijos para construir el Regex
    # Carpeta en URL: zonificacion_municipios | zonificacion_distritos | zonificacion_GAU
    folder_suffix = "GAU" if zone_type == "gau" else zone_type
    
    # Sufijo en ficheros CSV: nombres_municipios | nombres_distritos | nombres_gaus
    file_suffix = "gaus" if zone_type == "gau" else zone_type
    
    # --- REGEX PATTERNS ---
    # 1. Pattern para componentes del Shapefile (.shp, .shx, .dbf, .prj)
    # Busca URLs que contengan /zonificacion_{Suffix}/ y terminen en extensión de shapefile
    shp_pattern = rf'(https?://[^\s"<>]*/zonificacion/zonificacion_{folder_suffix}/[^"<>]+\.(?:shp|shx|dbf|prj))'
    
    # 2. Pattern para CSVs auxiliares (nombres_*.csv, poblacion_*.csv)
    # Busca URLs que contengan /zonificacion_{Suffix}/ y sean nombres_X.csv o poblacion_X.csv
    csv_pattern = rf'(https?://[^\s"<>]*/zonificacion/zonificacion_{folder_suffix}/(?:nombres|poblacion)_{file_suffix}\.csv)'

    print(f"📡 Scanning RSS for {zone_type} zoning files...")

    try:
        # Fetch RSS with User-Agent
        req = urllib.request.Request(rss_url, headers={"User-Agent": "MITMA-DuckLake-Loader"})
        with urllib.request.urlopen(req) as response:
            txt = response.read().decode("utf-8", "ignore")
        
        # Find matches
        shp_matches = re.findall(shp_pattern, txt, re.IGNORECASE)
        csv_matches = re.findall(csv_pattern, txt, re.IGNORECASE)
        
        # Deduplicate
        unique_shp = sorted(list(set(shp_matches)))
        unique_csv = sorted(list(set(csv_matches)))
        
        # Organizar resultados
        url_nombres = next((u for u in unique_csv if 'nombres' in u.lower()), None)
        url_poblacion = next((u for u in unique_csv if 'poblacion' in u.lower()), None)
        
        if not unique_shp and not unique_csv:
            print("WARNING: No zoning URLs found in RSS. The feed might have rotated them out.")
            # Opcional: Aquí podrías lanzar error o devolver fallback. 
            # Si quieres mantener el estilo estricto del otro script, devolvemos vacio.
            return {}

        print(f"Found {len(unique_shp)} shapefile components and {len(unique_csv)} CSVs.")
        
        return {
            "shp_components": unique_shp,
            "nombres": url_nombres,
            "poblacion": url_poblacion
        }

    except Exception as e:
        print(f"ERROR fetching RSS: {e}")
        return {}


In [18]:
def clean_id(series):
    """Normaliza ID a string limpio (sin .0, sin espacios)."""
    return series.astype(str).str.strip().str.replace(r'\.0$', '', regex=True)

def clean_poblacion(series):
    """Limpia enteros de población (quita puntos y decimales)."""
    return (series.astype(str)
            .str.replace('.', '', regex=False)
            .str.replace(r'\.0$', '', regex=True)
            .apply(pd.to_numeric, errors='coerce')
            .fillna(0).astype(int))

def get_mitma_zoning_dataset(zone_type='municipios'):
    """
    Orquesta la descarga, limpieza y fusión de datos maestros.
    Retorna un GeoDataFrame listo para ingesta.
    """
    urls = get_mitma_zoning_urls(zone_type)
    
    print(f"🚀 Generando dataset maestro para: {zone_type.upper()}")
    
    with tempfile.TemporaryDirectory() as tmp_dir:
        print("   ⬇️  Descargando geometrías...")
        shp_local_path = None
        
        for url in urls['shp_components']:
            filename = url.split('/')[-1]
            try:
                r = requests.get(url, timeout=15)
                if r.status_code == 200:
                    local_p = os.path.join(tmp_dir, filename)
                    with open(local_p, 'wb') as f:
                        f.write(r.content)
                    if filename.endswith('.shp'):
                        shp_local_path = local_p
            except Exception as e:
                print(f"      ⚠️ Error bajando {filename}: {e}")

        if not shp_local_path:
            print("❌ Error: No se pudo descargar el archivo .shp principal.")
            return None

        gdf = gpd.read_file(shp_local_path)
        
        id_col = next((c for c in gdf.columns if c.upper() in ['ID', 'CODIGO', 'ZONA', 'COD_GAU']), 'ID')
        gdf['ID'] = clean_id(gdf[id_col])
        
        gdf['geometry'] = gdf['geometry'].apply(make_valid)
        if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
            gdf = gdf.to_crs("EPSG:4326")

        print("   🔗 Integrando metadatos (Nombres y Población)...")
        df_aux = pd.DataFrame(columns=['ID'])
        
        aux_config = [
            {
                'type': 'nombres', 
                'url': urls['nombres'], 
                'header': 0, 
                'cols': ['ID', 'Nombre']
            },
            {
                'type': 'poblacion', 
                'url': urls['poblacion'], 
                'header': None, 
                'cols': ['ID', 'Poblacion']
            }
        ]

        for cfg in aux_config:
            try:
                r = requests.get(cfg['url'], timeout=10)
                if r.status_code == 200:
                    # Leer CSV crudo
                    df_t = pd.read_csv(
                        io.BytesIO(r.content), 
                        sep='|', 
                        header=cfg['header'], 
                        dtype=str, 
                        engine='python'
                    )
                    

                    if len(df_t.columns) >= 3:
                        df_t = df_t.iloc[:, [1, 2]]
                    elif len(df_t.columns) == 2:
                        df_t = df_t.iloc[:, [0, 1]]
                    
                    df_t.columns = cfg['cols']
                    
                    df_t['ID'] = clean_id(df_t['ID'])
                    df_t = df_t.drop_duplicates(subset=['ID'])
                    
                    if cfg['type'] == 'poblacion':
                        df_t['Poblacion'] = clean_poblacion(df_t['Poblacion'])

                    if df_aux.empty:
                        df_aux = df_t
                    else:
                        df_aux = df_aux.merge(df_t, on='ID', how='outer')
                        
                    print(f"      ✓ {cfg['type'].capitalize()} OK")
            except Exception as e:
                print(f"      ⚠️ Fallo procesando {cfg['type']}: {e}")

        # --- C. Merge Final ---
        if not df_aux.empty:
            gdf = gdf.merge(df_aux, on='ID', how='left')
            
            if 'Nombre' in gdf.columns: 
                gdf['Nombre'] = gdf['Nombre'].fillna(gdf['ID'])
            if 'Poblacion' in gdf.columns: 
                gdf['Poblacion'] = gdf['Poblacion'].fillna(0).astype(int)

        cols = ['ID', 'Nombre', 'Poblacion', 'geometry']
        final_cols = [c for c in cols if c in gdf.columns] + [c for c in gdf.columns if c not in cols]
        gdf = gdf[final_cols]

        print(f"✅ Dataset generado: {len(gdf)} registros.")
        return gdf

In [19]:
def load_zonificacion(type):
    """
    Load zonification data into DuckDB for the specified type.
    """
    df = get_mitma_zoning_dataset(type)
    
    if df is None or df.empty:
        print(f"No data to load for {type}")
        return
    
    # Convert all columns to string (including geometry)
    for col in df.columns:
        df[col] = df[col].astype(str)
    
    table_name = f'{LAKE_LAYER}_mitma_{type}'
    
    con.register('temp_zonificacion', df)
    
    SQL(f"""
        CREATE TABLE IF NOT EXISTS {table_name} AS
        SELECT
            *,
            CURRENT_TIMESTAMP AS loaded_at,
        FROM temp_zonificacion
        LIMIT 0;
    """)
    
    merge_key = 'ID'
    
    SQL(f"""
        MERGE INTO {table_name} AS target
        USING (
            SELECT
                *,
                CURRENT_TIMESTAMP AS loaded_at,
            FROM temp_zonificacion
        ) AS source
        ON target.{merge_key} = source.{merge_key}
        WHEN MATCHED THEN
            UPDATE SET *
        WHEN NOT MATCHED THEN
            INSERT *;
    """)
    
    con.unregister('temp_zonificacion')
    
    print(f"Table {table_name} merged successfully with {len(df)} records.")
    

In [20]:
SQL("""
    SELECT *
    FROM bronze_mitma_gau
    LIMIT 10
""")

,ID,Nombre,Poblacion,geometry,loaded_at
0,01001,Alegría-Dulantzi,29250,POINT (-2.511272 42.829065),2025-12-02 23:28:43.392995+01:00
1,01002,Amurrio,103070,POINT (-2.971689 43.025464),2025-12-02 23:28:43.392995+01:00
2,01004_AM,Artziniega agregacion de municipios,30050,POINT (-3.076283 43.150032),2025-12-02 23:28:43.392995+01:00
3,01009_AM,Asparrena agregacion de municipios,45990,POINT (-2.430988 42.883162),2025-12-02 23:28:43.392995+01:00
4,01010,Ayala/Aiara,29510,POINT (-3.078215 43.075551),2025-12-02 23:28:43.392995+01:00
5,01017_AM,Campezo/Kanpezu agregacion de municipios,43140,POINT (-2.435706 42.700123),2025-12-02 23:28:43.392995+01:00
6,01028_AM,Labastida/Bastida agregacion de municipios,75150,POINT (-2.687343 42.60083),2025-12-02 23:28:43.392995+01:00
7,01036,Laudio/Llodio,180090,POINT (-2.977515 43.13818),2025-12-02 23:28:43.392995+01:00
8,01043,Oyón-Oion,34180,POINT (-2.432799 42.544579),2025-12-02 23:28:43.392995+01:00
9,01047_AM,Erriberabeitia agregacion de municipios,37710,POINT (-3.074379 42.826092),2025-12-02 23:28:43.392995+01:00


In [21]:
table_name = "mitma_ine_relations"
# create_and_merge_table(table_name, ["https://movilidad-opendata.mitma.es/zonificacion/relacion_ine_zonificacionMitma.csv"])

In [22]:
SQL("""
    SELECT *
    FROM bronze_mitma_ine_relations
    LIMIT 10
""")

,seccion_ine,distrito_ine,municipio_ine,distrito_mitma,municipio_mitma,gau_mitma,loaded_at,source_file
0,0100101001,0100101,01001,01001,01001,01001,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
1,0100101002,0100101,01001,01001,01001,01001,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
2,0100201001,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
3,0100201002,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
4,0100201003,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
5,0100201004,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
6,0100201005,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
7,0100201006,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
8,0100201007,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
9,0100301001,0100301,01003,01058_AM,01058_AM,01058_AM,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...


In [23]:
def load_municipios_ine():
    """
    Load municipios from INE datasource
    """
    table_name = 'ine_municipios'
    url = 'https://servicios.ine.es/wstempus/js/ES/VALORES_VARIABLE/19'
    
    # Use 'Id' as the primary key for municipios
    create_and_merge_table_from_json(
        table_name, 
        url,
        ['Id']  # Assuming 'Id' is the unique identifier
    )

In [ ]:
def load_empresas_municipio_ine(year = 2023):
    """
    Load empresas from INE datasource. TOTAL OK
    """
    table_name = 'ine_empresas_municipio'
    url = f'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/4721?date={year}0101:{year}1231&Tv=40621:248341&Tv=selCri_2:on'
    
    # Use 'Id' as the primary key for municipios
    create_and_merge_table_from_json(
        table_name, 
        url,
        ['COD']  # Assuming 'Id' is the unique identifier
    )

In [ ]:
def load_poblacion_municipio_ine(year = 2023):
    """
    Load poblacion municipio from INE datasource
    """
    table_name = 'ine_poblacion_municipio'
    url = f'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/29005?date={year}0101:{year}1231&nult=1&det=2'
    
    # Use 'Id' as the primary key for municipios
    create_and_merge_table_from_json(
        table_name, 
        url,
        ['COD']  # Assuming 'Id' is the unique identifier
    ) 


In [135]:
def load_renta_municipio_ine(year = 2023):
    """
    Load empresas from INE datasource
    """
    table_name = 'ine_renta_municipio'
    # https://servicios.ine.es/wstempus/js/ES/TABLAS_OPERACION/353   -> Indicadores de renta media y mediana
    base_ids = [
        30656, 30833, 30842, 30851, 30860, 30869, 30878, 30887, 30896,
        30917, 30926, 30935, 30944, 30953, 30962, 30971, 30980, 30989, 30998,
        31007, 31016, 31025, 31034, 31043, 31052, 31061, 31070, 31079, 31088,
        31097, 31106, 31115, 31124, 31133, 31142, 31151, 31160, 31169, 31178,
        31187, 31196, 31205, 31214, 31223, 31232, 31241, 31250, 31259, 31268,
        31277, 31286, 31295
    ]

    urls = []
    for id in base_ids:
        urls.append(f'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/{id}?date={year}0101')
    
    # Use 'Id' as the primary key for municipios
    create_and_merge_table_from_json(
        table_name, 
        urls,
        ['COD']  # Assuming 'Id' is the unique identifier
    ) 


In [136]:
# load_municipios_ine()
# load_empresas_municipio_ine()
# load_poblacion_municipio_ine()
load_renta_municipio_ine()

Fetching JSON data from 52 URL(s)...
Using merge keys: ['COD']
Table bronze_ine_renta_municipio merged successfully with 1 key columns. Total rows: 354093


In [137]:
SQL("""
    SELECT *
    FROM bronze_ine_renta_municipio
""")

,COD,Nombre,FK_Unidad,FK_Escala,Data,loaded_at,source_url
0,ADRH90937,Ababuj. Dato base. Renta neta media por persona.,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
1,ADRH90936,Ababuj. Dato base. Renta neta media por hogar.,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
2,ADRH9816087,Ababuj. Dato base. Media de la renta por unida...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
3,ADRH9578257,Ababuj. Dato base. Mediana de la renta por uni...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
4,ADRH9578255,Ababuj. Dato base. Renta bruta media por perso...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
...,...,...,...,...,...,...,...
354088,ADRH10256780,Méntrida sección 01003. Dato base. Renta neta ...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
354089,ADRH10256785,Méntrida sección 01003. Dato base. Media de la...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
354090,ADRH10256781,Méntrida sección 01003. Dato base. Mediana de ...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
354091,ADRH10256148,Méntrida sección 01003. Dato base. Renta bruta...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-03 23:23:33.136247+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...


In [29]:
# MITMA data insertion

# import gc

# start_date = '2022-03-01'
# end_date = '2022-03-07'
# types = ['distritos', 'municipios', 'gau']

# for type in types:
#      """
#      Load OD matrices, people day, and overnight stay data for a specific type and date range.
#      """
#      load_od_matrices(type=type, start_date=start_date, end_date=end_date)
#      load_people_day(type=type, start_date=start_date, end_date=end_date)
#      load_overnight_stay(type=type, start_date=start_date, end_date=end_date)
#      load_zonificacion(type=type)
#      print(f"--- Liberando memoria tras {type} ---")
#      gc.collect()  # Fuerza al recolector de basura de Python a limpiar objetos no usados


# load_municipios_ine()
# load_empresas_municipio_ine()
# load_poblacion_municipio_ine()
# load_renta_municipio_ine()

In [30]:
print(SQL("SHOW TABLES;"))

                                      name
0                      bronze_ine_empresas
1            bronze_ine_empresas_municipio
2                    bronze_ine_municipios
3           bronze_ine_poblacion_municipio
4               bronze_ine_renta_municipio
5                   bronze_mitma_distritos
6                         bronze_mitma_gau
7               bronze_mitma_ine_relations
8         bronze_mitma_mitma_ine_relations
9                  bronze_mitma_municipios
10               bronze_mitma_od_distritos
11                     bronze_mitma_od_gau
12              bronze_mitma_od_municipios
13   bronze_mitma_overnight_stay_distritos
14         bronze_mitma_overnight_stay_gau
15  bronze_mitma_overnight_stay_municipios
16       bronze_mitma_people_day_distritos
17             bronze_mitma_people_day_gau
18      bronze_mitma_people_day_municipios
19                          ine_municipios
20             silver_mitma_overnight_stay
21                 silver_mitma_people_day


In [31]:
SQL("""
    CREATE OR REPLACE TABLE silver_mitma_people_day AS 
    WITH mitma_people_day_distritos AS (
        SELECT
            'distritos' as mitma_type,
            strptime(fecha::VARCHAR, '%Y%m%d') as fecha,
            zona_pernoctacion as pernoctacion_id,
            CAST(personas AS DOUBLE) AS personas,
            * EXCLUDE (fecha, zona_pernoctacion, personas, loaded_at, source_file)
        FROM bronze_mitma_people_day_distritos
    ), mitma_people_day_municipios AS (
        SELECT
            'municipios' as mitma_type,
            strptime(fecha::VARCHAR, '%Y%m%d') as fecha,
            zona_pernoctacion as pernoctacion_id,
            CAST(personas AS DOUBLE) AS personas,
            * EXCLUDE (fecha, zona_pernoctacion, personas, loaded_at, source_file)
        FROM bronze_mitma_people_day_municipios
    ), mitma_people_day_gau AS (
        SELECT
            'gau' as mitma_type,
            strptime(fecha::VARCHAR, '%Y%m%d') as fecha,
            zona_pernoctacion as pernoctacion_id,
            CAST(personas AS DOUBLE) AS personas,
            * EXCLUDE (fecha, zona_pernoctacion, personas, loaded_at, source_file)
        FROM bronze_mitma_people_day_gau
    )
    SELECT * FROM mitma_people_day_distritos 
    UNION ALL
    SELECT * FROM mitma_people_day_municipios 
    UNION ALL
    SELECT * FROM mitma_people_day_gau
""")

,Count
0,754845


In [32]:
SQL("""
    SELECT
        * 
    FROM silver_mitma_people_day
    LIMIT 10
""")

,mitma_type,fecha,pernoctacion_id,personas,edad,sexo,numero_viajes
0,municipios,2023-03-01,01001,187.481,0-25,hombre,0
1,municipios,2023-03-01,01001,83.147,0-25,hombre,2
2,municipios,2023-03-01,01001,213.587,0-25,hombre,2+
3,municipios,2023-03-01,01001,127.856,0-25,mujer,0
4,municipios,2023-03-01,01001,14.206,0-25,mujer,1
5,municipios,2023-03-01,01001,56.824,0-25,mujer,2
6,municipios,2023-03-01,01001,189.732,0-25,mujer,2+
7,municipios,2023-03-01,01001,93.458,25-45,hombre,0
8,municipios,2023-03-01,01001,73.008,25-45,hombre,2
9,municipios,2023-03-01,01001,169.274,25-45,hombre,2+


In [33]:
SQL("""
    CREATE OR REPLACE TABLE silver_mitma_overnight_stay AS 
    WITH mitma_overnight_stay_distritos AS (
        SELECT
            'distritos' as mitma_type,
            strptime(fecha::VARCHAR, '%Y%m%d') as fecha,
            zona_pernoctacion as pernoctacion_id,
            zona_residencia as residencia_id,
            CAST(personas AS DOUBLE) AS personas,
            * EXCLUDE (fecha, zona_pernoctacion, zona_residencia, personas, loaded_at, source_file)
        FROM bronze_mitma_overnight_stay_distritos
    ), mitma_overnight_stay_municipios AS (
        SELECT
            'municipios' as mitma_type,
            strptime(fecha::VARCHAR, '%Y%m%d') as fecha,
            zona_pernoctacion as pernoctacion_id,
            zona_residencia as residencia_id,
            CAST(personas AS DOUBLE) AS personas,
            * EXCLUDE (fecha, zona_pernoctacion, zona_residencia, personas, loaded_at, source_file)
        FROM bronze_mitma_overnight_stay_municipios
    ), mitma_overnight_stay_gau AS (
        SELECT
            'gau' as mitma_type,
            strptime(fecha::VARCHAR, '%Y%m%d') as fecha,
            zona_pernoctacion as pernoctacion_id,
            zona_residencia as residencia_id,
            CAST(personas AS DOUBLE) AS personas,
            * EXCLUDE (fecha, zona_pernoctacion, zona_residencia, personas, loaded_at, source_file)
        FROM bronze_mitma_overnight_stay_gau
    )
    SELECT * FROM mitma_overnight_stay_distritos 
    UNION ALL
    SELECT * FROM mitma_overnight_stay_municipios 
    UNION ALL
    SELECT * FROM mitma_overnight_stay_gau
""")

,Count
0,2426641


In [34]:
SQL("""
    SELECT
        * 
    FROM silver_mitma_overnight_stay
    LIMIT 10
""")

,mitma_type,fecha,pernoctacion_id,residencia_id,personas
0,distritos,2023-03-01,28134,38040,4.132
1,distritos,2023-03-01,2816101,38040,4.274
2,distritos,2023-03-01,2906702,38040,5.003
3,distritos,2023-03-01,29901,38040,4.554
4,distritos,2023-03-01,30009,38040,3.968
5,distritos,2023-03-01,33016,38040,8.313
6,distritos,2023-03-01,35003_AM,38040,4.274
7,distritos,2023-03-01,35008,38040,3.968
8,distritos,2023-03-01,35010,38040,11.558
9,distritos,2023-03-01,3501602,38040,5.003


In [35]:
SQL("""
    SELECT
        * 
    FROM bronze_mitma_ine_relations
    LIMIT 70
""")

,seccion_ine,distrito_ine,municipio_ine,distrito_mitma,municipio_mitma,gau_mitma,loaded_at,source_file
0,0100101001,0100101,01001,01001,01001,01001,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
1,0100101002,0100101,01001,01001,01001,01001,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
2,0100201001,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
3,0100201002,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
4,0100201003,0100201,01002,01002,01002,01002,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
...,...,...,...,...,...,...,...,...
65,0105701001,0105701,01057,01028_AM,01028_AM,01028_AM,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
66,0105801001,0105801,01058,01058_AM,01058_AM,01058_AM,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
67,0105901001,0105901,01059,0105901,01059,GAU Vitoria/Gasteiz,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...
68,0105901003,0105901,01059,0105901,01059,GAU Vitoria/Gasteiz,2025-12-02 20:53:25.217176+01:00,https://movilidad-opendata.mitma.es/zonificaci...


In [78]:
SQL("""
    SELECT count(*)
    FROM bronze_ine_municipios
""")

,count_star()
0,8192


In [101]:
SQL("""
    SELECT *
    FROM bronze_ine_poblacion_municipio
""")

,COD,Nombre,Unidad,Escala,Data,loaded_at,source_url
0,DPOP19723,Ababuj. Total. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
1,DPOP19724,Ababuj. Hombres. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
2,DPOP19725,Ababuj. Mujeres. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
3,DPOP17671,Abades. Total. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
4,DPOP17672,Abades. Hombres. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
...,...,...,...,...,...,...,...
24409,DPOP14696,Zúñiga. Hombres. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
24410,DPOP14697,Zúñiga. Mujeres. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
24411,DPOP1150,Zurgena. Total. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
24412,DPOP1151,Zurgena. Hombres. Total habitantes. Personas.,"{'Id': 3, 'Nombre': 'Personas', 'Codigo': None...","{'Id': 1, 'Nombre': ' ', 'Factor': '1E0', 'Cod...","[{'Fecha': 1704063600000, 'TipoDato': {'Id': 1...",2025-12-02 23:27:48.140783+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...


In [100]:
SQL("""
    SELECT *
    FROM bronze_ine_poblacion_municipio
    WHERE nombre = 'València' 
""")

,COD,Nombre,Unidad,Escala,Data,loaded_at,source_url


In [123]:
# EMPRESAS OK
SQL("""
    WITH empresas_flat AS (
        SELECT 
            e.COD,
            CAST(epoch_ms(CAST(data_item.Fecha AS BIGINT)) AS DATE) + INTERVAL 1 DAY AS fecha,
            COALESCE(NULLIF(TRIM(split_part(e.Nombre, '.', 1)), ''), e.Nombre) AS nombre,
            TRIM(split_part(e.Nombre, '.', 4)) AS tipo,
            CAST(data_item.Valor AS DOUBLE) AS valor
        FROM bronze_ine_empresas_municipio e,
            UNNEST(e.Data) AS t(data_item)
        WHERE e.Data IS NOT NULL 
          AND len(e.Data) > 0
          AND length(e.COD) = 9
    )
    SELECT DISTINCT ON (ef.COD)
        m.Codigo AS codigo_ine,
        ef.* EXCLUDE (COD),
        r.distrito_mitma,
        r.municipio_mitma,
        r.gau_mitma
    FROM empresas_flat ef
    LEFT JOIN bronze_ine_municipios m 
        ON ef.Nombre ILIKE m.Nombre
    LEFT JOIN bronze_mitma_ine_relations r
        ON m.Codigo = r.municipio_ine
    WHERE ef.Tipo ILIKE '%CNAE%'
    LIMIT 10
""")

,codigo_ine,fecha,nombre,tipo,valor,distrito_mitma,municipio_mitma,gau_mitma
0,05167,2023-01-01,Navarrevisca,Total CNAE,12.0,05054_AM,05054_AM,05054_AM
1,17202,2023-01-01,Tossa de Mar,Total CNAE,535.0,17202,17202,GAU Blanes - Lloret de Mar
2,46116,2023-01-01,"Eliana, l'",Total CNAE,1614.0,46116,46116,46116
3,50012,2023-01-01,Alborge,Total CNAE,9.0,50240_AM,50240_AM,50240_AM
4,34005,2023-01-01,Alar del Rey,Total CNAE,63.0,34083_AM,34083_AM,34083_AM
5,47085,2023-01-01,Medina del Campo,Total CNAE,1155.0,4708503,47085,47085
6,09085,2023-01-01,Castrillo de la Vega,Total CNAE,33.0,09141_AM,09141_AM,09141_AM
7,10039,2023-01-01,Cadalso,Total CNAE,20.0,10146_AM,10146_AM,10146_AM
8,19156,2023-01-01,Jadraque,Total CNAE,117.0,19156_AM,19156_AM,19156_AM
9,05256,2023-01-01,Villaflor,Total CNAE,NaN,05220_AM,05220_AM,05220_AM


In [ ]:
# POBLACION OK

SQL("""
    WITH poblacion_flat AS (
        SELECT 
            p.COD,
            CAST(epoch_ms(CAST(data_item.Fecha AS BIGINT)) AS DATE) + INTERVAL 1 DAY AS fecha,
            COALESCE(NULLIF(TRIM(split_part(p.Nombre, '.', 1)), ''), p.Nombre) AS nombre,
            LOWER(TRIM(split_part(p.Nombre, '.', 2))) AS tipo,
            CAST(data_item.Valor AS DOUBLE) AS valor
        FROM bronze_ine_poblacion_municipio p,
            UNNEST(p.Data) AS t(data_item)
        WHERE p.Data IS NOT NULL 
          AND len(p.Data) > 0
    )
    SELECT DISTINCT ON (pf.COD)
        m.Codigo AS codigo_ine,
        pf.* EXCLUDE (COD),
        r.distrito_mitma,
        r.municipio_mitma,
        r.gau_mitma
    FROM poblacion_flat pf
    LEFT JOIN bronze_ine_municipios m 
        ON pf.Nombre ILIKE m.Nombre
    LEFT JOIN bronze_mitma_ine_relations r
        ON m.Codigo = r.municipio_ine
    WHERE (
        r.distrito_mitma IS NOT NULL 
        OR r.municipio_mitma IS NOT NULL 
        OR r.gau_mitma IS NOT NULL
        )
    ORDER BY pf.COD ASC
""")

,codigo_ine,fecha,nombre,tipo,valor,distrito_mitma,municipio_mitma,gau_mitma
0,03056,2024-01-01,Cocentaina,total,11411.0,03056,03056,GAU Alcoy/Alcoi
1,03056,2024-01-01,Cocentaina,hombres,5755.0,03056,03056,GAU Alcoy/Alcoi
2,03056,2024-01-01,Cocentaina,mujeres,5656.0,03056,03056,GAU Alcoy/Alcoi


In [66]:
SQL("""
    SELECT *
    FROM bronze_ine_renta_municipio
    LIMIT 10
""")

,COD,Nombre,FK_Unidad,FK_Escala,Data,loaded_at,source_url
0,ADRH102301,Abrera. Dato base. Renta neta media por persona.,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
1,ADRH102300,Abrera. Dato base. Renta neta media por hogar.,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
2,ADRH9821769,Abrera. Dato base. Media de la renta por unida...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
3,ADRH9612349,Abrera. Dato base. Mediana de la renta por uni...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
4,ADRH9612348,Abrera. Dato base. Renta mediana por hogar.,7,1,[],2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
5,ADRH9612347,Abrera. Dato base. Renta bruta media por perso...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
6,ADRH9612346,Abrera. Dato base. Renta bruta media por hogar.,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
7,ADRH75855,Abrera distrito 01. Dato base. Renta neta medi...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
8,ADRH75854,Abrera distrito 01. Dato base. Renta neta medi...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...
9,ADRH9807923,Abrera distrito 01. Dato base. Media de la ren...,7,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2025-12-02 23:27:22.354134+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...


In [62]:
SQL("""
WITH renta_flat AS (
    SELECT 
        r.COD,
        CAST(epoch_ms(CAST(data_item.Fecha AS BIGINT)) AS DATE) + INTERVAL 1 DAY AS fecha,
        -- Limpiar el nombre: extraer solo el nombre de la población
        TRIM(regexp_replace(
            split_part(r.Nombre, '.', 1),
            '\s+(distrito|sección|seccion)\s+\d+',
            '',
            'gi'
        )) AS nombre,
        LOWER(TRIM(split_part(r.Nombre, '.', 3))) AS tipo,
        CAST(data_item.Valor AS DOUBLE) AS valor,
        -- Extraer el tipo de entidad geográfica
        CASE 
            WHEN regexp_matches(LOWER(r.Nombre), '(sección|seccion)\s+\d+') THEN 'seccion'
            WHEN regexp_matches(LOWER(r.Nombre), 'distrito\s+\d+') THEN 'distrito'
            ELSE 'municipio'
        END AS tipo_ine,
        -- Extraer el código de sección/distrito si existe
        CASE 
            WHEN regexp_matches(LOWER(r.Nombre), '(sección|seccion)\s+\d+') THEN 
                TRIM(regexp_extract(r.Nombre, 'secci[oó]n\s+(\d+)', 1))
            WHEN regexp_matches(LOWER(r.Nombre), 'distrito\s+\d+') THEN 
                TRIM(regexp_extract(r.Nombre, 'distrito\s+(\d+)', 1))
            ELSE NULL
        END AS codigo_zona
    FROM bronze_ine_renta_municipio r,
        UNNEST(r.Data) AS t(data_item)
    WHERE r.Data IS NOT NULL 
      AND len(r.Data) > 0
)
SELECT 
    m.Codigo AS codigo_ine,
    rf.* EXCLUDE (COD),
    rel.distrito_mitma,
    rel.municipio_mitma,
    rel.gau_mitma
FROM renta_flat rf
LEFT JOIN bronze_ine_municipios m 
    ON rf.nombre ILIKE m.Nombre
LEFT JOIN bronze_mitma_ine_relations rel
    ON m.Codigo = rel.municipio_ine
WHERE rf.tipo_ine = 'seccion'
LIMIT 10
""")

,codigo_ine,fecha,nombre,tipo,valor,tipo_ine,codigo_zona,distrito_mitma,municipio_mitma,gau_mitma
0,08001,2023-01-01,Abrera,renta neta media por persona,16065.0,seccion,01001,08001,08001,GAU Barcelona
1,08001,2023-01-01,Abrera,mediana de la renta por unidad de consumo,20650.0,seccion,01002,08001,08001,GAU Barcelona
2,08003,2023-01-01,Alella,renta neta media por hogar,62267.0,seccion,01001,08003,08003,GAU Barcelona
3,08003,2023-01-01,Alella,media de la renta por unidad de consumo,42793.0,seccion,01003,08003,08003,GAU Barcelona
4,08003,2023-01-01,Alella,mediana de la renta por unidad de consumo,33250.0,seccion,01003,08003,08003,GAU Barcelona
5,08004,2023-01-01,Alpens,renta bruta media por hogar,54327.0,seccion,01001,08149_AM,08149_AM,08149_AM
6,08005,2023-01-01,"Ametlla del Vallès, L'",media de la renta por unidad de consumo,35327.0,seccion,01003,08005,08005,GAU Barcelona
7,08005,2023-01-01,"Ametlla del Vallès, L'",mediana de la renta por unidad de consumo,31150.0,seccion,01003,08005,08005,GAU Barcelona
8,08005,2023-01-01,"Ametlla del Vallès, L'",media de la renta por unidad de consumo,34591.0,seccion,01004,08005,08005,GAU Barcelona
9,08006,2023-01-01,Arenys de Mar,renta neta media por persona,16570.0,seccion,01001,0800602,08006,GAU Barcelona


In [157]:
SQL("""
    WITH renta_flat AS (
        SELECT 
            r.COD,
            CAST(epoch_ms(CAST(data_item.Fecha AS BIGINT)) AS DATE) + INTERVAL 1 DAY AS fecha,
            -- Limpiar el nombre: extraer solo el nombre de la población
            TRIM(regexp_replace(
                split_part(r.Nombre, '.', 1),
                '\s+(distrito|sección|seccion)\s+\d+',
                '',
                'gi'
            )) AS nombre,
            LOWER(TRIM(split_part(r.Nombre, '.', 3))) AS tipo,
            CAST(data_item.Valor AS DOUBLE) AS valor
        FROM bronze_ine_renta_municipio r,
            UNNEST(r.Data) AS t(data_item)
        WHERE r.Data IS NOT NULL 
        AND len(r.Data) > 0
        -- Filtrar solo registros de municipio (sin distrito ni sección)
        AND NOT regexp_matches(LOWER(r.Nombre), '(sección|seccion)\s+\d+')
        AND NOT regexp_matches(LOWER(r.Nombre), 'distrito\s+\d+')
    )
    SELECT DISTINCT ON (rf.COD)
        m.Codigo AS codigo_ine,
        rf.* EXCLUDE (COD),
        rel.distrito_mitma,
        rel.municipio_mitma,
        rel.gau_mitma
    FROM renta_flat rf
    LEFT JOIN bronze_ine_municipios m 
        ON rf.nombre ILIKE m.Nombre
    LEFT JOIN bronze_mitma_ine_relations rel
        ON m.Codigo = rel.municipio_ine
    WHERE valor IS NOT NULL AND (
        distrito_mitma IS NOT NULL 
        OR municipio_mitma IS NOT NULL 
        OR gau_mitma IS NOT NULL
    ) AND rf.nombre = 'Alacant/Alicante'
    ORDER by codigo_ine ASC
""")

,codigo_ine,fecha,nombre,tipo,valor,distrito_mitma,municipio_mitma,gau_mitma
0,03014,2023-01-01,Alacant/Alicante,mediana de la renta por unidad de consumo,17150.0,0301408,03014,GAU Alicante - Elche
1,03014,2023-01-01,Alacant/Alicante,renta neta media por hogar,34936.0,0301408,03014,GAU Alicante - Elche
2,03014,2023-01-01,Alacant/Alicante,renta bruta media por persona,16858.0,0301408,03014,GAU Alicante - Elche
3,03014,2023-01-01,Alacant/Alicante,renta bruta media por hogar,43048.0,0301408,03014,GAU Alicante - Elche
4,03014,2023-01-01,Alacant/Alicante,media de la renta por unidad de consumo,20118.0,0301408,03014,GAU Alicante - Elche
5,03014,2023-01-01,Alacant/Alicante,renta neta media por persona,13681.0,0301408,03014,GAU Alicante - Elche
